In [3]:
# This code is from Qiskit Hackathon 2021 by the team
# Qiskit for high dimensional multipartite quantum states.
#
# Author: Hoang Van Do
#
# (C) Copyright 2021 Hoang Van Do, Tim Alexis Körner, Inho Choi, Timothé Presles and Élie Gouzien.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.exceptions import QiskitError

#Pi coupling between m and l level. The inverse of the LevelsSwitch function is itself
def level_switch(m, l, dimension):
    if m > dimension or l > dimension:
        raise QiskitError('The level is higher than the dimension')
    n=int(np.ceil(np.log2(dimension)))
    qreg =QuantumRegister(n)
    areg = AncillaRegister(1)
    circuit=QuantumCircuit(qreg, areg)
    control_qubits = qreg[:]
    target_qubit = areg[0]

    #save indices of qubits which are 1 for states m, l
    marray=[]
    larray=[]
    for i in range(n):
        if (m >>  i) & 1 != 1:
            marray.append(i)
    for i in range(n):
        if (l >>  i) & 1 != 1:
            larray.append(i)

    #control on m, l
    if len(marray)>0:
        circuit.x(marray)
    circuit.mcx(control_qubits,target_qubit)
    if len(marray)>0:
        circuit.x(marray)
    if len(larray)>0:
        circuit.x(larray)
    circuit.mcx(control_qubits,target_qubit)
    if len(larray)>0:
        circuit.x(larray)
    
    #swap
    for i in range(n):
        if (( m >>  i) & 1) != (( l >>  i) & 1):
            circuit.cx(n, i)
    
    #control on m, l to reset auxiliary qubit    
    if len(marray) > 0:
        circuit.x(marray)
    circuit.mcx(control_qubits,target_qubit)
    if len(marray) > 0:
        circuit.x(marray)
    if len(larray) > 0:
        circuit.x(larray)
    circuit.mcx(control_qubits,target_qubit)
    if len(larray) > 0:
        circuit.x(larray)
    
    return circuit

from qiskit import Aer, execute
backend = Aer.get_backend('unitary_simulator')
np.set_printoptions(linewidth=200, precision=2, suppress=True)

qc = level_switch(2, 3, 8)
print(qc)
job = execute(qc, backend)
result = job.result()
U = result.get_unitary(qc)

N = int(U.shape[0]/2)
print("Auxiliary qubit should start and end in state |0> (only look at top left of matrix)")
print(U[:N,:N])




       ┌───┐     ┌───┐               ┌───┐┌───┐     ┌───┐               
q12_0: ┤ X ├──■──┤ X ├───────■───────┤ X ├┤ X ├──■──┤ X ├───────■───────
       └───┘  │  └───┘       │       └─┬─┘└───┘  │  └───┘       │       
q12_1: ───────■──────────────■─────────┼─────────■──────────────■───────
       ┌───┐  │  ┌───┐┌───┐  │  ┌───┐  │  ┌───┐  │  ┌───┐┌───┐  │  ┌───┐
q12_2: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├──┼──┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├
       └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘  │  └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘
 a1_0: ─────┤ X ├──────────┤ X ├───────■───────┤ X ├──────────┤ X ├─────
            └───┘          └───┘               └───┘          └───┘     
Auxiliary qubit should start and end in state |0> (only look at top left of matrix)
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 